# Travail de NLP sur des décisions juridiques

[Projet issu de ce lien :](https://challengedata.ens.fr/challenges/24)

Import des librairies

In [1]:
import numpy as np
import pandas as pd
import os
import re
import codecs

[Outil pour le Regex :](https://pythex.org/)

In [42]:
regex_date = "([1]?[0-9]/[0-1][0-9]/[1-2][0-9]{3}|[1]?[0-9]-[0-1][0-9]-[1-2][0-9]{3})"
regex_date_litteral = "([0-9]{1,2} *(janvier|f[e|é]vrier|mars|avril|mai|juin|juillet|ao[u|û]t|septembre|octobre|novembre|d[e|é]cembre) *[0-9]{2,4})"

Définition des chemins d'accès

In [3]:
train_folder = "train_folder_predilex/train_folder/"
test_folder = "test_folder_predilex/test_folder/"
train_path = "train_folder_predilex/train_folder/txt_files/"
test_path = "test_folder_predilex/test_folder/txt_files/"

Appel des .csv en DataFrame

In [4]:
df_train_y = pd.read_csv("y_train_predilex.csv")
df_train_x = pd.read_csv("train_folder_predilex/train_folder/x_train_ids.csv")
df_train = df_train_x.set_index('ID').join(df_train_y.set_index('ID'))
#df_train.head()

In [5]:
df_test_y = pd.read_csv("y_test_pred.csv")
df_test_x = pd.read_csv("test_folder_predilex/test_folder/x_test_ids.csv")
df_test = df_test_x.set_index('ID').join(df_test_y.set_index('filename'))
#df_test.head()

Mise en place d'un filtre de lecture des fichiers texte

In [6]:
def read_properly(path,filename):
    f = codecs.open(os.path.join(path,filename),"r",encoding='utf-8',errors="replace")
    return f.read().replace('\xa0','').replace('\n',' ').replace('’',' ')\
                 .replace(',',' ').replace('...','___').replace('.',' ')\
                 .replace(':',' ').replace('(',' ').replace(')',' ')

Lecture des fichiers et ajout aux DataFrames respectifs

In [7]:
df_train['contenu'] = [read_properly(train_path,i) for i in df_train.filename]

In [8]:
df_test['contenu'] = [read_properly(test_path,i) for i in df_test.filename]

Fonction pour avoir la fréquence d'un mot dans une chaîne de caractères (+ WIP sur Regex)

In [9]:
def frequency_word(text,word):
    return sum(1 for i in text.split() if i.lower()==word)

In [10]:
def frequency_regex(text,regex):
    return re.findall(regex,text)

Recherche des mots de genre dans les textes respectifs

In [11]:
df_train['CC_elle']=[frequency_word(text,"elle") for text in df_train.contenu]
df_train['CC_il']=[frequency_word(text,"il") for text in df_train.contenu]
df_train['CC_madame']=[frequency_word(text,"madame") for text in df_train.contenu]
df_train['CC_monsieur']=[frequency_word(text,"monsieur") for text in df_train.contenu]
df_train['CC_mme']=[frequency_word(text,"mme") for text in df_train.contenu]
df_train['CC_m']=[frequency_word(text,"m") for text in df_train.contenu]

In [12]:
df_test['CC_elle']=[frequency_word(text,"elle") for text in df_test.contenu]
df_test['CC_il']=[frequency_word(text,"il") for text in df_test.contenu]
df_test['CC_madame']=[frequency_word(text,"madame") for text in df_test.contenu]
df_test['CC_monsieur']=[frequency_word(text,"monsieur") for text in df_test.contenu]
df_test['CC_mme']=[frequency_word(text,"mme") for text in df_test.contenu]
df_test['CC_m']=[frequency_word(text,"m") for text in df_test.contenu]

Ajout des comptes de mots de même genre pour cumul total

In [13]:
df_train['Homme'] = df_train.CC_monsieur+df_train.CC_m+df_train.CC_il
df_train['Femme'] = df_train.CC_madame+df_train.CC_mme+df_train.CC_elle

In [14]:
df_test['Homme'] = df_test.CC_monsieur+df_test.CC_m+df_test.CC_il
df_test['Femme'] = df_test.CC_madame+df_test.CC_mme+df_test.CC_elle

Détermination du genre prédominant dans le texte

In [15]:
df_train['Genre'] = ['homme' if df_train.Homme.tolist()[i]>df_train.Femme.tolist()[i] else\
                     'femme' if df_train.Homme.tolist()[i]<df_train.Femme.tolist()[i] else\
                     'neutre' for i in range(df_train.shape[0])]
df_test['Genre'] = ['homme' if df_test.Homme.tolist()[i]>df_test.Femme.tolist()[i] else\
                     'femme' if df_test.Homme.tolist()[i]<df_test.Femme.tolist()[i] else\
                     'neutre' for i in range(df_test.shape[0])]

Précision de cette NLP comparé à la réalité

In [18]:
train_accuracy = np.sum(df_train.sexe==df_train.Genre)/df_train.shape[0]
test_accuracy = np.sum(df_test.sexe==df_test.Genre)/df_test.shape[0]
"Train",train_accuracy*100,"Test",test_accuracy*100

('Train', 89.74025974025975, 'Test', 64.59143968871595)

In [19]:
df_train.head()

,filename,sexe,date_accident,date_consolidation,contenu,CC_elle,CC_il,CC_madame,CC_monsieur,CC_mme,CC_m,Homme,Femme,Genre
ID,,,,,,,,,,,,,,
0,Agen_100515.txt,homme,1991-04-09,n.c.,Le 12/11/2019 Cour d appel d Agen chambr...,5,11,10,22,0,1,34,15,homme
1,Agen_1100752.txt,homme,2005-06-10,2010-01-19,Le 12/11/2019 Cour d appel d Agen chambr...,3,60,9,65,0,0,125,12,homme
2,Agen_1613.txt,femme,1997-09-26,n.c.,Le 12/11/2019 Cour d appel d Agen Audien...,24,3,16,3,0,0,6,40,femme
3,Agen_2118.txt,femme,1982-08-07,1982-11-07,Le 12/11/2019 Cour d appel d Agen Audien...,11,16,1,2,0,1,19,12,homme
4,Agen_21229.txt,homme,1996-11-26,n.c.,Le 12/11/2019 Cour d appel d Agen Audien...,1,32,1,1,0,32,65,2,homme


In [43]:
a = df_train.contenu.tolist()[3]
frequency_regex(a,regex_date_litteral),frequency_regex(a,regex_date)

([('20 août 2003', 'août'),
  ('7 août 1982', 'août'),
  ('7 mars 1995', 'mars'),
  ('5 juillet 1985', 'juillet'),
  ('13 octobre 1995', 'octobre'),
  ('7 septembre 1999', 'septembre'),
  ('5 juillet 1985', 'juillet'),
  ('26 mars 1990', 'mars'),
  ('24 janvier 1996', 'janvier'),
  ('7 avril 1983', 'avril'),
  ('5 juillet 1985', 'juillet'),
  ('27 décembre 1990', 'décembre'),
  ('7 août 1982', 'août'),
  ('27 décembre 1990', 'décembre'),
  ('7 août 1982', 'août'),
  ('7 août 1982', 'août'),
  ('7 novembre 1982', 'novembre'),
  ('5 juillet 1985', 'juillet'),
  ('7 avril 1983', 'avril'),
  ('7 avril 1983', 'avril'),
  ('5 juillet 1985', 'juillet')],
 ['12/11/2019'])